In [378]:
import random
import openpyxl

In [379]:
N_COURSES = 15
# Indices > 9 are lab courses
courses = ['AI', 'PF', 'OOP', 'CAL', 'ENG', 'DLD', 'COAL', 'NC', 'PDC', 'SE', 'DATA-LAB', 'OS-LAB', 'CNET-LAB', 'WEB-LAB', 'AI-LAB']

# No of sections for each course
sections = [2, 5, 3, 5, 1, 3, 4, 2, 1, 5, 4, 3, 2, 2, 4]
sections_sum = 0
for  section in sections:
    sections_sum += section

# No of students in each section for each course
students_in_sections = []

for num_sections in sections:
    students_per_section = [random.randint(30, 120) for _ in range(num_sections)]
    students_in_sections.append(students_per_section)

# List of professors
professors = [
    "Ayesha Bano", "Aqeel Shahzad", "Farah Naz", "Hamda Khan", "Usman Rashid",
    "Farah Jabeen Awan", "Sara Aziz", "Gul e Aisha", "Maimoona Rassol", "Sajid Khan",
    "Tayyab Nadeem", "Mehboobullah", "Muhammad bin Qasim", "Zainab Moin", "Sumera Abbas",
    "Sadia Nauman", "Shahzad Mehmood", "Sanaa Ilyas", "Nagina Safdar", "Asma Nisa",
    "Sehrish Hassan", "Waqas Munir", "Usman Ashraf", "Muhammad Usman", "Naveed Ahmad",
    "Zainab Abaid", "Rohail Gulbaz", "Hassan Raza", "Javaria Imtiaz", "Maheen Arshad",
    "Zohaib Iqbal", "Mehreen Alam", "Mehwish Hassan", "Shams Farooq", "Ameen Chilwan",
    "Shafaq Riaz", "Khadija Farooq", "Amna Irum", "Irum Inayat", "Bilal Khalid",
    "Shehreyar Rashid", "Umair Arshad", "Subhan Ullah", "Ejaz Ahmed", "Kifayat Ullah"
]

# Dictionary to keep track of the number of courses each professor has been assigned
professor_course_count = {professor: 0 for professor in professors}

# Assign professors to course sections
course_details = []

for i, course in enumerate(courses):
    course_type = "Lab" if i > 9 else "Theory"
    for j in range(sections[i]):
        # Randomly select a professor who has been assigned less than 3 courses
        available_professors = [professor for professor, count in professor_course_count.items() if count < 3]
        selected_professor = random.choice(available_professors)
        section_name = f"{course} Section {j+1}"
        section_strength = students_in_sections[i][j]
        course_details.append([course, course_type, section_name, section_strength, selected_professor])
        # Increment the course count for the selected professor
        professor_course_count[selected_professor] += 1

# # Printing the results
# for course_detail in course_details:
#     print(course_detail)

# print(len(course_details))


days = ['Monday','Tuesday','Wednesday','Thursday','Friday']
time_slots = [
    ("08:30 AM", "09:50 AM"),
    ("10:05 AM", "11:25 AM"),
    ("11:40 AM", "01:00 PM"),
    ("01:15 PM", "02:35 PM"),
    ("02:50 PM", "04:10 PM"),
    ("04:25 PM", "05:45 PM")
]

rooms = [
    ('A_201', 60), ('A_202', 120), ('A_203', 60), ('A_204', 60), ('A_205', 120),
    ('A_206', 60), ('A_207', 120), ('A_208', 60), ('A_209', 120), ('A_210', 120),
    ('B_201', 120), ('B_202', 120), ('B_203', 60), ('B_204', 60), ('B_205', 120),
    ('B_206', 120), ('B_207', 120), ('B_208', 60), ('B_209', 60), ('B_210', 60),
    ('C_301', 120), ('C_302', 60), ('C_303', 120), ('C_304', 60), ('C_305', 60),
    ('C_306', 60), ('C_307', 120), ('C_308', 60), ('C_309', 120), ('C_310', 120),
    ('C_401', 120), ('C_402', 120), ('C_403', 60), ('C_404', 60), ('C_405', 60),
    ('C_406', 120), ('C_407', 60), ('C_408', 60), ('C_409', 120), ('C_410', 120),
]

INIT_POPULATION = 1000
GENE_LENGTH = 24
CHROMOSOME_LENGTH = GENE_LENGTH * sections_sum


In [380]:
def fitness_function(chromosome):
  # print('Length of genes',len(chromosome))
  
  clashes = 0
  # Extract all the genes from the chromosome
  genes = [chromosome[i:i+GENE_LENGTH] for i in range(0, len(chromosome), GENE_LENGTH)]
  # print('Length of genes',len(genes))

  for i in range(len(genes)):
    gene1 = genes[i]

    # Extracting indices for gene1
    gene1_day_1 = int(gene1[:3], 2)
    gene1_time_1 = int(gene1[3:6], 2)
    gene1_room_1 = int(gene1[6:12], 2)
    gene1_day_2 = int(gene1[12:15], 2)
    gene1_time_2 = int(gene1[15:18], 2)
    gene1_room_2 = int(gene1[18:], 2)

    # CHECKS NOT APPLICABLE TO LABS
    if course_details[i][1] != "Lab":
      #if both classes for course are assigned on same day
      if gene1_day_1 == gene1_day_2:
        clashes -=1
        # print('clash of course ',course_details[i][2], ' with itself same day class ',days[gene1_day_1], ' ', days[gene1_day_2])

      #if both classes for course are on adjacent day
      if abs(gene1_day_1 - gene1_day_2) == 1:
        clashes -=1
        # print('clash of course ',course_details[i][2], ' with itself adjacent class',days[gene1_day_1], ' ', days[gene1_day_2])

      # if not same room for both sessions, soft constraint so -0.5
      if gene1_room_1 != gene1_room_2:
        clashes -= 0.5
        
      # if class is in evening session
      if gene1_time_1 > 3:
        clashes -= 0.5
      
      if gene1_time_2 > 3:
        clashes -= 0.5
    
    # if lab is in morning session
    if course_details[i][1] == "Lab" and gene1_time_1 < 4:
        clashes -= 0.5
    # if rooms are not big enough
    if course_details[i][3] > rooms[gene1_room_1][1]:
      clashes -=1
      # print('clash of course ',course_details[i][2], ' with itself room not big on day 1 ', ' Strength ', course_details[i][3], ' room size ',rooms[gene1_room_1][1])


    if course_details[i][3] > rooms[gene1_room_2][1]:
      clashes -=1
      # print('clash of course ',course_details[i][2], ' with itself room not big on day 2 ', ' Strength ', course_details[i][3], ' room size ',rooms[gene1_room_2][1])

    # print('I mein hn ',course_details[i][2])
    # print('I k din ',days[gene1_day_1],days[gene1_day_2])

    for j in range(i+1, len(genes)):
        gene2 = genes[j]

        # Extracting indices for gene2
        gene2_day_1 = int(gene2[:3], 2)
        gene2_time_1 = int(gene2[3:6], 2)
        gene2_room_1 = int(gene2[6:12], 2)
        gene2_day_2 = int(gene2[12:15], 2)
        gene2_time_2 = int(gene2[15:18], 2)
        gene2_room_2 = int(gene2[18:], 2)

        # print('j mein hn ',course_details[j][2])
        
        # print('J k din ',days[gene2_day_1],days[gene2_day_2])


        # IF 'I' IS LAB AND 'J' IS THEORY
        if course_details[i][1] == "Lab" and course_details[j][1] == "Theory":
          #if class is on same day and same slot
          if gene1_day_1 == gene2_day_1 and (gene1_time_1 == gene2_time_1 or gene1_time_2 == gene2_time_1):
            #if same professor in both sections
            if course_details[i][4] == course_details[j][4]:
              # print('here kesy ', gene1_day_1 , gene2_day_1)           

              clashes -=1
              # print('LAB theroy clash ',course_details[i][2],course_details[j][2],days[gene1_day_1], days[gene2_day_1], time_slots[gene1_time_1] , time_slots[gene2_time_1] , time_slots[gene1_time_2] , time_slots[gene2_time_1])
              
              # print('Professor clash')
            #if same room assigned to both sections
            if gene1_room_1 == gene2_room_1:
              # print('here kesy ', gene1_day_1 , gene2_day_1)           

              clashes -=1
              # print('LAB theroy clash ',course_details[i][2],course_details[j][2],days[gene1_day_1], days[gene2_day_1], time_slots[gene1_time_1] , time_slots[gene2_time_1] , time_slots[gene1_time_2] , time_slots[gene2_time_1])
              
              # print('Room clash')
              
              
          #if class is on same day and same slot
          if gene1_day_1 == gene2_day_2 and (gene1_time_1 == gene2_time_2 or gene1_time_2 == gene2_time_2):
            
            #if same professor in both sections
            if course_details[i][4] == course_details[j][4]:
            #   print('here kesy ', gene1_day_1 , gene2_day_2)           

              clashes -=1
            #   print('LAB theroy clash ',course_details[i][2],course_details[j][2],days[gene1_day_1], days[gene2_day_2], time_slots[gene1_time_1] , time_slots[gene2_time_2] , time_slots[gene1_time_2] , time_slots[gene2_time_2])
              
            #   print('Professor clash')
              
            # #if same room assigned to both sections
            if gene1_room_1 == gene2_room_2:
            #   print('here kesy ', gene1_day_1 , gene2_day_2)           

              clashes -=1
            #   print('LAB theroy clash ',course_details[i][2],course_details[j][2],days[gene1_day_1], days[gene2_day_2], time_slots[gene1_time_1] , time_slots[gene2_time_2] , time_slots[gene1_time_2] , time_slots[gene2_time_2])
              
            #   print('Room clash')
              
        
        # IF 'J' IS LAB AND 'I' IS THEORY
        elif course_details[i][1] == "Theory" and course_details[j][1] == "Lab":
          #if class is on same day and same slot
          if gene2_day_1 == gene1_day_1 and (gene2_time_1 == gene1_time_1 or gene2_time_2 == gene1_time_1): 
            #if same professor in both sections
            if course_details[i][4] == course_details[j][4]:
              # print('here kesy ', gene2_day_1 , gene1_day_1)           

              clashes -=1
              # print('theroy lab clash ',course_details[i][2],course_details[j][2],days[gene2_day_1], days[gene1_day_1], time_slots[gene2_time_1] , time_slots[gene1_time_1] , time_slots[gene2_time_2] , time_slots[gene1_time_1])
              
              # print('Professor clash')
              
            #if same room assigned to both sections
            if gene2_room_1 == gene1_room_1:
              # print('here kesy ', gene2_day_1 , gene1_day_1)           

              clashes -=1
              # print('theroy lab clash ',course_details[i][2],course_details[j][2],days[gene2_day_1], days[gene1_day_1], time_slots[gene2_time_1] , time_slots[gene1_time_1] , time_slots[gene2_time_2] , time_slots[gene1_time_1])
              
              # print('Room clash')
              
              
          #if class is on same day and same slot
          if gene2_day_1 == gene1_day_2 and (gene2_time_1 == gene1_time_2 or gene2_time_2 == gene1_time_2):
            
            #if same professor in both sections
            if course_details[i][4] == course_details[j][4]:
              # print('here kesy ', gene2_day_1 , gene1_day_2)           
              clashes -=1
              # print('theroy lab clash ',course_details[i][2],course_details[j][2],days[gene2_day_1], days[gene1_day_2], time_slots[gene2_time_1] , time_slots[gene1_time_2] , time_slots[gene2_time_2] , time_slots[gene1_time_2])
              
              # print('Professor clash')
              
            #if same room assigned to both sections
            if gene2_room_1 == gene1_room_2:
              # print('here kesy ', gene2_day_1 , gene1_day_2)           

              clashes -=1
              # print('theroy lab clash ',course_details[i][2],course_details[j][2],days[gene2_day_1], days[gene1_day_2], time_slots[gene2_time_1] , time_slots[gene1_time_2] , time_slots[gene2_time_2] , time_slots[gene1_time_2])
              
              # print('Room clash')
              
              
        # IF BOTH ARE LABS 
        elif course_details[i][1] == "Lab" and course_details[j][1] == "Lab":
          #if class is on same day and same slot
          if gene2_day_1 == gene1_day_1 :
            if gene2_time_1 == gene1_time_1 or gene2_time_1 == gene1_time_2 or gene2_time_2 == gene1_time_1 or gene2_time_2 == gene1_time_2: 
              #if same professor in both sections
              if course_details[i][4] == course_details[j][4]:
                # print('here kesy ', gene2_day_1 , gene1_day_1)           

                clashes -=1
                # print('lab lab clash ',course_details[i][2],course_details[j][2],days[gene2_day_1], days[gene1_day_1], time_slots[gene2_time_1] , time_slots[gene1_time_1] , time_slots[gene2_time_1] , time_slots[gene1_time_2] , time_slots[gene2_time_2] , time_slots[gene1_time_1] , time_slots[gene2_time_2] , time_slots[gene1_time_2])
                # print('Professor clash')
                
              #if same room assigned to both sections
              if gene2_room_1 == gene1_room_1:
                # print('here kesy ', gene2_day_1 , gene1_day_1)           

                clashes -=1
                # print('lab lab clash ',course_details[i][2],course_details[j][2],days[gene2_day_1], days[gene1_day_1], time_slots[gene2_time_1] , time_slots[gene1_time_1] , time_slots[gene2_time_1] , time_slots[gene1_time_2] , time_slots[gene2_time_2] , time_slots[gene1_time_1] , time_slots[gene2_time_2] , time_slots[gene1_time_2])
                # print('Room clash')
                
        # IF BOTH ARE THEORY    
        else: 
          #if same slot on same day
          if gene1_day_1 == gene2_day_1 and gene1_time_1 == gene2_time_1:
            #if same professor in both sections
            if course_details[i][4] == course_details[j][4]:
              clashes -=1
              # print('clash of course ',course_details[i][2], ' with ',course_details[j][2] , ' Professor1 ', course_details[i][4], ' Professor2 ',course_details[j][4], ' g1day1 ',days[gene1_day_1],' g2day1 ',days[gene2_day_1], ' g1Time1 ', time_slots[gene1_time_1], ' g2Time1 ', time_slots[gene2_time_1] )
            #if same room assigned to both sections
            if gene1_room_1 == gene2_room_1:
              clashes -=1
              # print('clash of course ',course_details[i][2], ' with ',course_details[j][2] , ' Room1 ', rooms[gene1_room_1], ' Room2 ',rooms[gene2_room_1], ' g1day1 ',days[gene1_day_1],' g2day1 ',days[gene2_day_1], ' g1Time1 ', time_slots[gene1_time_1], ' g2Time1 ', time_slots[gene2_time_1] )

          #if same slot on same day
          if gene1_day_1 == gene2_day_2 and gene1_time_1 == gene2_time_2:
            #if same professor in both sections
            if course_details[i][4] == course_details[j][4]:
              clashes -=1
              # print('clash of course ',course_details[i][2], ' with ',course_details[j][2] , ' Professor1 ', course_details[i][4], ' Professor2 ',course_details[j][4], ' g1day1 ',days[gene1_day_1],' g2day2 ',days[gene2_day_2], ' g1Time1 ', time_slots[gene1_time_1], ' g2Time2 ', time_slots[gene2_time_2] )
            #if same room assigned to both sections
            if gene1_room_1 == gene2_room_2:
              clashes -=1
              # print('clash of course ',course_details[i][2], ' with ',course_details[j][2] , ' Room1 ', rooms[gene1_room_1], ' Room2 ',rooms[gene2_room_2], ' g1day1 ',days[gene1_day_1],' g2day2 ',days[gene2_day_2], ' g1Time1 ', time_slots[gene1_time_1], ' g2Time2 ', time_slots[gene2_time_2] )
          #if same slot on same day
          if gene1_day_2 == gene2_day_1 and gene1_time_2 == gene2_time_1:
            #if same professor in both sections
            if course_details[i][4] == course_details[j][4]:
              clashes -=1
              # print('clash of course ',course_details[i][2], ' with ',course_details[j][2] , ' Professor1 ', course_details[i][4], ' Professor2 ',course_details[j][4], ' g1day2 ',days[gene1_day_2],' g2day1 ',days[gene2_day_1], ' g1Time2 ', time_slots[gene1_time_2], ' g2Time1 ', time_slots[gene2_time_1] )
            #if same room assigned to both sections
            if gene1_room_2 == gene2_room_1:
              clashes -=1
              # print('clash of course ',course_details[i][2], ' with ',course_details[j][2] , ' Room1 ', rooms[gene1_room_2], ' Room2 ',rooms[gene2_room_1], ' g1day2 ',days[gene1_day_2],' g2day1 ',days[gene2_day_1], ' g1Time2 ', time_slots[gene1_time_2], ' g2Time1 ', time_slots[gene2_time_1] )
          #if same slot on same day
          if gene1_day_2 == gene2_day_2 and gene1_time_2 == gene2_time_2:
            #if same professor in both sections
            if course_details[i][4] == course_details[j][4]:
              clashes -=1
              # print('clash of course ',course_details[i][2], ' with ',course_details[j][2] , ' Professor1 ', course_details[i][4], ' Professor2 ',course_details[j][4], ' g1day2 ',days[gene1_day_2],' g2day2 ',days[gene2_day_2], ' g1Time2 ', time_slots[gene1_time_2], ' g2Time2 ', time_slots[gene2_time_2] )
            #if same room assigned to both sections
            if gene1_room_2 == gene2_room_2:
              clashes -=1
              # print('clash of course ',course_details[i][2], ' with ',course_details[j][2] , ' Room1 ', rooms[gene1_room_2], ' Room2 ',rooms[gene2_room_2], ' g1day2 ',days[gene1_day_2],' g2day2 ',days[gene2_day_2], ' g1Time2 ', time_slots[gene1_time_2], ' g2Time2 ', time_slots[gene2_time_2] )

  return clashes

In [381]:
def tournament_selection(chromosomes,fitness_scores):
    # we've to play INIT_POPULATION Rounds to select new parents for crossover, no tossing on tie, first one is selected
    selected_chromosomes = []
    for _ in range(INIT_POPULATION):
        canditate1 = random.randint(0,INIT_POPULATION -1)
        canditate2 = random.randint(0,INIT_POPULATION -1)
        # print(canditate1,' VS ',canditate2)
        if fitness_scores[canditate1] >= fitness_scores[canditate2]:
            selected_chromosomes.append(chromosomes[canditate1])
            # print('Winner ',canditate1)
        else:
            selected_chromosomes.append(chromosomes[canditate2])
            # print('Winner ',canditate2)
            
    return selected_chromosomes

In [382]:
def mutate(chromosome):
    # print('Old chromosome:', chromosome)
    new_chromosome = list(chromosome)  # Convert the string to a list
    point = random.randint(0, len(chromosome) - 1)  # Generate a random mutation point
    
    # Perform mutation
    if new_chromosome[point] == '0':
        new_chromosome[point] = '1'
    else:
        new_chromosome[point] = '0'

    new_chromosome = ''.join(new_chromosome)  # Convert the list back to a string
    # print('New chromosome:', new_chromosome)
    return new_chromosome

In [383]:
def correct_the_genes(chromosome):
    # Extract all the genes from the chromosome
    genes = [chromosome[i:i+GENE_LENGTH] for i in range(0, len(chromosome), GENE_LENGTH)]
    
    # Iterate through each gene
    for i in range(len(genes)):
        gene = genes[i]
        
        # Extracting indices for the gene
        day_1 = int(gene[:3], 2)
        time_1 = int(gene[3:6], 2)
        room_1 = int(gene[6:12], 2)
        day_2 = int(gene[12:15], 2)
        time_2 = int(gene[15:18], 2)
        room_2 = int(gene[18:], 2)
        
        # Check and correct day_1
        if day_1 > len(days) -1 :
            # print('Masla ')
            day_1 = random.randint(0, len(days) -1)
            if course_details[i][1] == "Lab":
                # print('Aur lab ka')
                day_2 = day_1
                # print('Day ',day_1,day_2)
        
        # Check and correct time_1
        if time_1 > len(time_slots) -1:
            # print('Masla ')
            time_1 = random.randint(0, len(time_slots) -1)
            if course_details[i][1] == "Lab":
                # print('Aur lab ka')
                time_1 = random.randint(0, len(time_slots) -2)
                time_2 = time_1 + 1
                # print('time ',time_1,time_2)
        
        # For lab
        if course_details[i][1] == "Lab" and time_1 > len(time_slots) -2:
            time_1 = random.randint(0,4)

        
        # Check and correct room_1
        if room_1 > len(rooms) -1:
            # print('Masla ')
            room_1 = random.randint(0, len(rooms) -1)
            if course_details[i][1] == "Lab":
                # print('Aur lab ka')
                room_2 = room_1
                
        
        # Check and correct day_2
        if day_2 > len(days) -1:
            day_2 = random.randint(0, len(days) -1)
            if course_details[i][1] == "Lab":
                # print('Aur lab ka')
                day_2 = day_1
                
        
        # Check and correct time_2
        if time_2 > len(time_slots) -1:
            time_2 = random.randint(0, len(time_slots) -1)
            if course_details[i][1] == "Lab":
                # print('Aur lab ka')
                time_2 = time_1 + 1
                
        
        # Check and correct room_2
        if room_2 > len(rooms) -1:
            room_2 = random.randint(0, len(rooms) -1)
            if course_details[i][1] == "Lab":
                # print('Aur lab ka')
                room_2 = room_1
                # print('room ',room_1,room_2)
                
            
        if course_details[i][1] == "Lab":
            day_2 = day_1
            time_2 = time_1 + 1
            room_2 = room_1
            # print('Masly ka hal')
            # print('Professor ',course_details[i][4])
            # print('day ', days[day_1])
            # print('time ',time_slots[time_1])
            # print('room ', rooms[room_1])
            # print('day ', days[day_2])
            # print('time ',time_slots[time_2])
            # print('room ', rooms[room_2])
        
        # Update the gene
        genes[i] = format(day_1, '03b') + format(time_1, '03b') + format(room_1, '06b') + format(day_2, '03b') + format(time_2, '03b') + format(room_2, '06b')
    
    # Reconstruct the chromosome with corrected genes
    corrected_chromosome = ''.join(genes)
    
    return corrected_chromosome


In [384]:
def two_point_crossover(parent1,parent2):
    # print('Parent1 ',parent1)
    # print('Parent2 ',parent2)
    
    child1 = ''
    child2 = ''
    points = random.sample(range(1, CHROMOSOME_LENGTH), 2) # takes two distinct random points
    point1 = min(points)
    point2 = max(points)
    
    # point1 = 2
    # point2 = 5
    
    # print('point1 ',point1)
    # print('point2 ',point2)
    
    
    child1 = parent1[:point1]
    child1 += parent2[point1:point2]
    child1 += parent1[point2:]
    
    child2 = parent2[:point1]
    child2 += parent1[point1:point2]
    child2 += parent2[point2:]
    
    # print('child1 ',child1)
    # print('child2 ',child2)
    
    return child1,child2

In [385]:
def generate_initial_population():
    # Define the lengths of each component in the gene
    day_length = 3
    time_slot_length = 3
    room_length = 6

    chromosomes = []
    for _ in range(INIT_POPULATION):
        chromosome = ''
        i=0
        # print('------------------------------ CHROMOSOME ',i+1,' -----------------------------------')
        for course_detail in course_details:
            # print('course ',course_details[i][2])
            # Randomly select indices for days, time_slots, and rooms
            day_index_1 = -1
            room_index_1 = -1
            day_index_2 = -1
            room_index_2 = -1
            time_slot_index_1 = -1
            time_slot_index_2 = -1 
            # don't assign last slot to labs
            if course_detail[1] == "Lab":
                time_slot_index_1 = random.randint(0, len(time_slots) - 2) 
                time_slot_index_2 = time_slot_index_1 + 1
                day_index_1 = random.randint(0, len(days) - 1)
                day_index_2 = day_index_1
                room_index_1 = random.randint(0, len(rooms) - 1)
                room_index_2 = room_index_1      
            else:
                time_slot_index_1 = random.randint(0, len(time_slots) - 1)       
                time_slot_index_2 = random.randint(0, len(time_slots) - 1)       
                day_index_1 = random.randint(0, len(days) - 1)
                room_index_1 = random.randint(0, len(rooms) - 1)
                day_index_2 = random.randint(0, len(days) - 1)
                room_index_2 = random.randint(0, len(rooms) - 1)      
                

            # print('Professor ',course_details[i][4])
            # print('day ', days[day_index_1])
            # print('time ',time_slots[time_slot_index_1])
            # print('room ', rooms[room_index_1])
            # print('day ', days[day_index_2])
            # print('time ',time_slots[time_slot_index_2])
            # print('room ', rooms[room_index_2])
            i += 1


            # Convert indices to binary strings
            day_binary_1 = format(day_index_1, f'0{day_length}b')
            time_slot_binary_1 = format(time_slot_index_1, f'0{time_slot_length}b')
            room_binary_1 = format(room_index_1, f'0{room_length}b')
            day_binary_2 = format(day_index_2, f'0{day_length}b')
            time_slot_binary_2 = format(time_slot_index_2, f'0{time_slot_length}b')
            room_binary_2 = format(room_index_2, f'0{room_length}b')

            # Append the gene to the chromosome
            chromosome += day_binary_1 + time_slot_binary_1 + room_binary_1 + day_binary_2 + time_slot_binary_2 + room_binary_2
            # print('\n')

        chromosomes.append(chromosome)
    
    return chromosomes

In [386]:
def get_fitness_scores_of_population(population):
    fitness_scores = []
    maqsad = False
    maqsad_chromosome = ''
    
    for chromosome in population:
        # print("Chromosome:", chromosome)
        fitness_score = fitness_function(chromosome)
        if fitness_score == 0:
            print('Maqsad Acheived')
            maqsad = True
            maqsad_chromosome = chromosome
        fitness_scores.append(fitness_score)
        # print(fitness_function(chromosome))
    
    return fitness_scores,maqsad,maqsad_chromosome

In [387]:
def perform_crossover_and_mutation(population):
    new_generation = []

    for i in range(0, len(population) - 1, 2):
        child1, child2 = two_point_crossover(population[i],population[i+1])
        
        child1 = mutate(child1)
        child2 = mutate(child2)
    
        child1 = correct_the_genes(child1)
        child2 = correct_the_genes(child2)
        
        new_generation.append(child1)
        new_generation.append(child2)
        
    return new_generation

In [388]:
def print_timetable(chromosome):
    # Extract all the genes from the chromosome
    genes = [chromosome[i:i+GENE_LENGTH] for i in range(0, len(chromosome), GENE_LENGTH)]
    
    # Iterate through each gene
    for i in range(len(genes)):
        gene = genes[i]
        
        # Extracting indices for the gene
        day_1 = int(gene[:3], 2)
        time_1 = int(gene[3:6], 2)
        room_1 = int(gene[6:12], 2)
        day_2 = int(gene[12:15], 2)
        time_2 = int(gene[15:18], 2)
        room_2 = int(gene[18:], 2)
        
        print('course ',course_details[i][2])
        print('Professor ',course_details[i][4])
        print('day ', days[day_1])
        print('time ',time_slots[time_1])
        print('room ', rooms[room_1])
        print('day ', days[day_2])
        print('time ',time_slots[time_2])
        print('room ', rooms[room_2])
        print('\n')
        
    #     file_name = "timetable.xlsx"
    
    # # Check if the file exists
    # if os.path.exists(file_name):
    #     # If the file exists, remove it
    #     os.remove(file_name)

In [389]:
def create_timetable_excel(chromosome):
    genes = [chromosome[i:i+GENE_LENGTH] for i in range(0, len(chromosome), GENE_LENGTH)]
    
    # Create a new workbook
    wb = openpyxl.Workbook()
    
    # Remove default sheet
    wb.remove(wb.active)
    
    # Create a sheet for each day of the week
    for day in days:
        ws = wb.create_sheet(title=day)
        for col in range(len(time_slots)):
            ws.cell(row=1, column=col+2, value=f"{time_slots[col]}")  # Adjust column index by +2
            # Set the width of each column to 40 pixels
            ws.column_dimensions[openpyxl.utils.get_column_letter(col+2)].width = 50  # Adjust column index by +2
        for row in range(len(rooms)):
            ws.cell(row=row+2, column=1, value=f"{rooms[row]}")
        # Set the width of the first column to 25 pixels
        ws.column_dimensions['A'].width = 25
    
    # # Iterate through each gene and place course details in the corresponding cells
    for i in range(len(genes)):
        gene = genes[i]
        # Extracting indices for the gene
        day_1 = int(gene[:3], 2)
        time_1 = int(gene[3:6], 2)
        room_1 = int(gene[6:12], 2)
        day_2 = int(gene[12:15], 2)
        time_2 = int(gene[15:18], 2)
        room_2 = int(gene[18:], 2)
        
        # Write course details to the cells for the first day and time slot
        ws1 = wb[days[day_1]]
        for row in range(2, len(rooms) + 2):  # Adjust row index by +2
            if ws1.cell(row=row, column=1).value == f"{rooms[room_1]}":  # Check room number
                # print('row value ',ws1.cell(row=row, column=1).value,' room value ', rooms[room_1] )
                ws1.cell(row=row, column=time_1 + 2, value=course_details[i][2]+ '  Professor: ' + course_details[i][4])  # Adjust column index by +2
                break  # Exit loop after finding the correct row
        
        # Write course details to the cells for the second day and time slot
        ws2 = wb[days[day_2]]
        for row in range(2, len(rooms) + 2):  # Adjust row index by +2
            if ws2.cell(row=row, column=1).value == f"{rooms[room_2]}":  # Check room number
                ws2.cell(row=row, column=time_2 + 2, value=course_details[i][2] + '  Professor: '+ course_details[i][4])  # Adjust column index by +2
                break  # Exit loop after finding the correct row

    # Save the workbook
    wb.save("timetable.xlsx")


In [390]:
# INITIAL_POPULATION
initial_population = generate_initial_population()
population  = initial_population
# print_timetable(population[0])
# create_timetable_excel(population[0])
# SET THE MAX ITERATIONS FOR GENETIC ALGORITHM
MAX_ITERATION = 1000

for i in range(MAX_ITERATION):
    print('------------------------------------------iteration------------------------------------------ ', i+1)
    # print_timetable(population[0])
    # Get fitness scores of population  
    fitness_scores,maqsad,maqsad_chromsome = get_fitness_scores_of_population(population)
    print(fitness_scores)
    if maqsad == True:
        print_timetable(maqsad_chromsome)
        create_timetable_excel(maqsad_chromsome)
        break

    # Perform the Tournament selection on population and select new parents for crossover
    selected_chromosomes = tournament_selection(population,fitness_scores)
    # print('selected_chromosomes ',selected_chromosomes)
    # Get the new generation after performing crossover and mutation
    new_generation = perform_crossover_and_mutation(selected_chromosomes)
    # print('new gent ', new_generation)
    population = new_generation


------------------------------------------iteration------------------------------------------  1
[-93.0, -83.0, -84.0, -103.0, -88.0, -88.5, -88.0, -88.5, -87.5, -98.0, -92.5, -103.5, -105.5, -88.0, -95.0, -79.0, -89.0, -93.5, -90.5, -90.5, -97.0, -98.5, -81.5, -89.0, -89.0, -109.0, -81.5, -89.0, -90.5, -89.0, -94.0, -91.0, -103.5, -85.5, -104.0, -92.0, -89.5, -91.0, -86.5, -94.0, -92.0, -97.0, -93.0, -89.5, -99.0, -89.5, -85.0, -91.5, -76.0, -81.5, -88.5, -90.5, -95.5, -78.5, -85.0, -97.5, -89.5, -87.5, -97.0, -94.5, -78.0, -84.5, -83.0, -93.0, -91.0, -92.5, -103.0, -89.0, -94.0, -84.0, -102.0, -88.0, -87.5, -87.0, -96.5, -91.5, -93.5, -95.0, -82.5, -95.0, -95.5, -77.5, -95.0, -69.5, -80.0, -93.0, -84.0, -81.5, -83.0, -94.0, -84.0, -95.0, -88.5, -95.5, -90.0, -88.5, -86.0, -88.5, -88.5, -96.5, -102.5, -92.0, -94.0, -81.0, -90.5, -92.0, -102.5, -92.5, -84.5, -103.5, -90.0, -83.0, -100.0, -96.5, -87.5, -86.0, -103.5, -82.5, -94.5, -85.0, -87.0, -87.0, -96.5, -88.0, -90.0, -87.5, -86.5, 